# Vector Search

This section will introduce you to a different way of searching that leverages Machine Learning (ML) techniques to interpret meaning and context.



## Introduction to Embeddings
In Machine Learning, an embedding is a vector (an array of numbers) that represents real-world objects such as words, sentences, images or videos. The interesting property that these embeddings have is that two embeddings that represent similar or related real-world entities will share some similarities as well, so embeddings can be compared, and a distance between them can be calculated.

When thinking specifically in terms of an application for searching, performing a search of embeddings in the vector space tends to find results that are more related to concepts, instead of to the exact keywords typed in the search prompt.

In this section of the tutorial you are going to learn how to generate embeddings using freely available machine learning models, then you will use Elasticsearch's vector database support to store and search these embeddings. And towards the end, you will also combine vector and full-text search results and create a powerful hybrid search solution that offers the best of both approaches.



## Generate Embeddings
In this section you are going to learn about one of the most convenient options that are available to generate embeddings for text, which is based on the SentenceTransformers framework.

Working with SentenceTransformers is the recommended path while you explore and become familiar with the use of embeddings, as the models that are available under this framework can be installed on your computer, perform reasonably well without a GPU, and are free to use.



## SentenceTransformers

SentenceTransformers는 문장이나 텍스트의 의미를 고차원 벡터 공간에 매핑하는 데 사용되는 Python 프레임워크입니다. 이 프레임워크는 주로 자연어 처리(NLP) 작업에 사용되며, 특히 Vector Search와 같은 의미 기반 검색에 매우 유용합니다.

기본 개념:
- 텍스트를 고정 길이의 dense vector로 변환합니다.
- 이 벡터는 텍스트의 의미적 내용을 포착합니다.

모델 기반:
- 주로 BERT, RoBERTa, XLM-RoBERTa 등의 트랜스포머 모델을 기반으로 합니다.
- 다양한 사전 훈련된 모델을 제공합니다.

다국어 지원:
- 100개 이상의 언어를 지원하는 모델을 포함합니다.

사용 예시: 

```python
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings = model.encode(sentences)
print(embeddings)
```




## Install SentenceTransformers
The SentenceTransformers framework is installed as a Python package. Make sure that your Python virtual environment is activated, and then run the following command on your terminal to install this framework:




In [1]:
%pip install sentence-transformers

  Using cached sentence_transformers-3.0.1-py3-none-any.whl.metadata (10 kB)
  Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 10.1 MB/s eta 0:00:0000:0100:01
Using cached scipy-1.13.1-cp39-cp39-macosx_12_0_arm64.whl (30.3 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Selecting a Model
The next task is to decide on a machine learning model to use for embedding generation. There is a list of pretrained models in the documentation. Because SentenceTransformers is a very popular framework, there are also compatible models created by researchers not directly associated with the framework. To see a complete list of models that can be used, you can check the SentenceTransformers tag on HuggingFace.

For the purposes of this tutorial there is no need to overthink the model selection, as any model will suffice. The SentenceTransformers documentation includes the following note with regards to their pretrained models:

The all-* models where trained on all available training data (more than 1 billion training pairs) and are designed as general purpose models. The all-mpnet-base-v2 model provides the best quality, while all-MiniLM-L6-v2 is 5 times faster and still offers good quality.

This seems to suggest that their all-MiniLM-L6-v2 model is a good choice that offers a good compromise between speed and quality, so let's use this model. Locate this model in the table, and click the "info" icon to see some information about it.

An interesting detail that is good to be aware of about your chosen model is the length the generated embeddings have, or in other words, how many numbers or dimensions the resulting vectors will have. This is important because it directly affects the amount of storage you will need. In the case of all-MiniLM-L6-v2, the generated vectors have 384 dimensions.



## Loading the Model
The following Python code demonstrates how the model is loaded. You can try this in a Python shell.




In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

/Users/jeongmin/PycharmProjects/python-skills-bootcamp/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/jeongmin/PycharmProjects/python-skills-bootcamp/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jeongmin/PycharmProjects/python-skills-bootcamp/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


The first time you do this, the model will be downloaded and installed in your virtual environment, so the call may take some time to return. Once the model is installed, instantiating it should not take long.



## Generating Embeddings
With the model instantiated, you are now ready to generate an embedding. To do this, pass the source text to the model.encode() method:




In [4]:
embedding = model.encode('The quick brown fox jumps over the lazy dog')

In [5]:
embedding

array([ 3.54968011e-02,  6.12862706e-02,  5.26920557e-02,  7.07050413e-02,
        3.31013948e-02, -3.06696109e-02,  6.62061898e-03, -6.11832850e-02,
       -1.32602104e-03,  1.06456643e-02,  3.86499353e-02,  3.99531797e-02,
       -3.83675545e-02, -1.66687984e-02, -5.61561156e-03, -2.43558995e-02,
       -3.59968394e-02, -3.02429274e-02,  5.84700331e-02, -4.94961701e-02,
       -7.72954598e-02, -5.23876920e-02,  2.45271791e-02,  2.93105822e-02,
       -7.39091635e-02, -2.49592010e-02, -6.53142035e-02, -4.28864807e-02,
        7.11656362e-02, -1.13819472e-01, -1.26593364e-02,  3.96260805e-02,
       -2.10036244e-02,  1.78064071e-02, -3.18874270e-02, -9.11229402e-02,
        5.91224581e-02, -7.30397413e-03,  3.31367701e-02,  2.99061127e-02,
        4.21688668e-02, -1.69129800e-02, -4.50015739e-02,  2.96744388e-02,
       -9.92584750e-02,  5.32891788e-02, -7.64784738e-02, -1.48679931e-02,
        1.52494945e-02,  1.37893632e-02, -4.41923775e-02, -2.78393030e-02,
        6.73079677e-03,  

The result is an array with all the numbers that make up the embedding. As you recall, the embeddings generated by the chosen model have 384 dimensions, so this is the length of the embedding array.



## Store Embeddings in Elasticsearch
Elasticsearch provides full support for storing and retrieving vectors, which makes it an ideal database for working with embeddings.



## Field Types
In the Full-Text Search chapter of this tutorial you have learned how to create an index with several fields. At that time it was mentioned that Elasticsearch can, for the most part, automatically determine the best type to use for each field based on the data itself. Even though Elasticsearch 8.11 is able to automatically map some vector types, in this chapter you will define this type explicitly as an opportunity to learn more about type mappings in Elasticsearch.



## Retrieving Type Mappings
The types associated with each field in an index are determined in a process called mapping, which can be dynamic or explicit. The mappings that were created in the Full-Text Search portion of this tutorial were all dynamically generated by Elasticsearch.

The Elasticsearch client offers a get_mapping method, which returns the type mappings that are in effect for a given index. If you want to explore these mappings on your own, start a Python shell and enter the following code:




In [6]:
import pprint
from elasticsearch import Elasticsearch

class Search:
    def __init__(self):
        self.es = Elasticsearch('http://localhost:9200')
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)

    def create_index(self):
        self.es.indices.delete(index='my_documents', ignore_unavailable=True)
        self.es.indices.create(index='my_documents', mappings={
            'properties': {
                'embedding': {
                    'type': 'dense_vector',
                }
            }
        })

    def insert_documents(self, documents):
        operations = []
        for document in documents:
            operations.append({'index': {'_index': 'my_documents'}})
            operations.append(document)
        return self.es.bulk(operations=operations)

    def reindex(self):
        self.create_index()
        with open('data.json', 'rt') as f:
            documents = json.loads(f.read())
        return self.insert_documents(documents)


## Dense vector field type

정의:
- dense_vector 필드 타입은 숫자 값의 밀집 벡터를 저장합니다.
- 주로 k-최근접 이웃(k-nearest neighbor, kNN) 검색에 사용됩니다.


제한사항:
- 이 필드 타입은 집계(aggregations)나 정렬(sorting)을 지원하지 않습니다.


구조:
- 숫자 값의 배열로 저장됩니다.
- 기본적으로 float 타입의 element_type을 사용합니다.

매핑 예시:
```json
"my_vector": {
  "type": "dense_vector",
  "dims": 3
}
```

- "dims": 3은 이 벡터가 3차원임을 나타냅니다.

사용 예시:
- 인덱스 생성 및 매핑 정의:

```json
PUT my-index
{
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "dense_vector",
        "dims": 3
      },
      "my_text" : {
        "type" : "keyword"
      }
    }
  }
}
```

문서 추가:
```json
PUT my-index/_doc/1
{
  "my_text" : "text1",
  "my_vector" : [0.5, 10, 6]
}

PUT my-index/_doc/2
{
  "my_text" : "text2",
  "my_vector" : [-0.5, 10, 10]
}
```

주의사항:
- 벡터의 차원 수(dims)는 매핑 시 명시적으로 정의해야 하며, 문서마다 일관된 차원 수를 유지해야 합니다.


## Index vectors for kNN search

kNN 검색 정의:
- 쿼리 벡터와 가장 유사한 k개의 벡터를 찾는 검색 방법입니다.

Dense vector 필드의 사용:
- script_score 쿼리에서 문서 랭킹에 사용될 수 있습니다.
- 모든 문서를 스캔하는 브루트 포스 kNN 검색이 가능합니다.

효율적인 kNN 검색:
- 브루트 포스 방식은 비효율적일 수 있어, 특수한 데이터 구조로 벡터를 인덱싱합니다.
- search API의 knn 옵션을 통해 빠른 kNN 검색을 지원합니다.

동적 매핑:
- 128에서 4096 사이의 크기를 가진 float 배열은 자동으로 dense_vector로 매핑됩니다.
- 기본 유사도 메트릭은 코사인 유사도입니다

인덱싱 비용:
- 벡터 인덱싱은 비용이 많이 드는 작업입니다.
- 인덱싱이 활성화된 벡터 필드가 있는 문서의 색인 생성에 상당한 시간이 걸릴 수 있습니다.

인덱싱 설정:
- 기본적으로 int8_hnsw로 인덱싱됩니다.
- 유사도 메트릭을 지정할 수 있습니다 (예: dot_product).

```json 
PUT my-index-2
{
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "dense_vector",
        "dims": 3,
        "similarity": "dot_product"
      }
    }
  }
}
```

인덱싱 비활성화:
- index 파라미터를 false로 설정하여 인덱싱을 비활성화할 수 있습니다.

```json
PUT my-index-2
{
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "dense_vector",
        "dims": 3,
        "index": false
      }
    }
  }
}
```

HNSW 알고리즘:
- Elasticsearch는 효율적인 kNN 검색을 위해 HNSW(Hierarchical Navigable Small World) 알고리즘을 사용합니다.
- 이는 근사 방법으로, 결과의 정확성을 일부 희생하여 검색 속도를 향상시킵니다.




## Automatically quantize vectors for kNN search

자동 벡터 양자화의 목적:
- float 벡터를 검색할 때 필요한 메모리 사용량을 줄이기 위해 사용됩니다.
- 원래는 dense vector 에서 각 벡터당 float 이니까 4바이트 씀. 여기서는 int8_hnsw 를 써서 1바이트로 줄인거고



지원되는 양자화 방법:
- 현재는 int8 양자화만 지원됩니다.
- 입력 벡터의 element_type은 반드시 float이어야 합니다.

int8_hnsw 인덱스의 특징:
- 각 float 벡터의 차원을 1바이트 정수로 양자화합니다.
- 메모리 사용량을 최대 75%까지 줄일 수 있습니다.
- 정확도가 약간 감소할 수 있습니다.
- 디스크 사용량은 양자화된 벡터와 원본 벡터를 모두 저장하기 때문에 25% 정도 증가할 수 있습니다.

```json
PUT my-byte-quantized-index
{
  "mappings": {
    "properties": {
      "my_vector": {
        "type": "dense_vector",
        "dims": 3,
        "index": true,
        "index_options": {
          "type": "int8_hnsw"
        }
      }
    }
  }
}
```





## Adding Embeddings to Documents
In the previous section you have learned how to generate embeddings using the SentenceTransformers framework and the all-MiniLM-L6-v2 model. Now it is time to integrate the model into the application.

First of all, the model can be instantiated in the Search class constructor:




In [7]:
import pprint
from elasticsearch import Elasticsearch

class Search:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.es = Elasticsearch('http://localhost:9200')
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)

    def create_index(self):
        self.es.indices.delete(index='my_documents', ignore_unavailable=True)
        self.es.indices.create(index='my_documents', mappings={
            'properties': {
                'embedding': {
                    'type': 'dense_vector',
                }
            }
        })

    def insert_documents(self, documents):
        operations = []
        for document in documents:
            operations.append({'index': {'_index': 'my_documents'}})
            operations.append(document)
        return self.es.bulk(operations=operations)

    def reindex(self):
        self.create_index()
        with open('data.json', 'rt') as f:
            documents = json.loads(f.read())
        return self.insert_documents(documents)


As you recall from the full-text search portion of this tutorial, the Search class has insert_document() and insert_documents() methods, to insert single and multiple documents into the index respectively. These two methods now need to generate the corresponding embeddings that go with each document.

The next code block shows new versions of these two methods, along with a new get_embedding() helper method that returns an embedding.




In [ ]:
import pprint
from elasticsearch import Elasticsearch

class Search:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.es = Elasticsearch('http://localhost:9200')
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        pprint(client_info.body)

    def create_index(self):
        self.es.indices.delete(index='my_documents', ignore_unavailable=True)
        self.es.indices.create(index='my_documents', mappings={
            'properties': {
                'embedding': {
                    'type': 'dense_vector',
                }
            }
        })

    def insert_document(self, document):
        return self.es.index(index='my_documents', document={
            **document,
            'embedding': self.get_embedding(document['summary']),
        })
    
    def insert_documents(self, documents):
        operations = []
        for document in documents:
            operations.append({'index': {'_index': 'my_documents'}})
            operations.append({
                **document,
                'embedding': self.get_embedding(document['summary']),
            })
        return self.es.bulk(operations=operations)


    def reindex(self):
        self.create_index()
        with open('data.json', 'rt') as f:
            documents = json.loads(f.read())
        return self.insert_documents(documents)
    
    def get_embedding(self, text):
        return self.model.encode(text)



The modified methods add the new embedding field to the document to be inserted. The embedding is generated from the summary field of each document. In general, embeddings are generated from sentences or short paragraphs, so in this case the summary is an ideal field to use. Other options would have been the name field, which contains the title of the document, or maybe the first few sentences from the document's body.

With these changes in place the index can be rebuilt, so that it stores an embedding for each document. To rebuilt the index, use this command:




## k-Nearest Neighbor (kNN) Search

The k-nearest neighbor (kNN) algorithm performs a similarity search on fields of dense_vector type. This type of search, which is more appropriately called "approximate kNN", accepts a vector or embedding as a search term, and finds entries in the index that are close.

In this section you are going to learn how to run a kNN search using the document embeddings created in the previous section.



## The knn Query
In the full-text search section of the tutorial you learned about the query option passed to the search() method of the Elasticsearch client. When searching vectors, the knn option is used instead.

Below you can see a new version of the handle_search() function in app.py that runs a kNN search for the query entered by the user in the search form.




## k-nearest neighbor (kNN) search

kNN 검색 정의:
- 쿼리 벡터와 가장 유사한 k개의 벡터를 찾는 검색 방법입니다.
- 유사도 메트릭을 사용하여 측정합니다.


kNN의 일반적인 사용 사례:
- 자연어 처리(NLP) 알고리즘 기반의 관련성 랭킹
- 제품 추천 및 추천 엔진
- 이미지나 비디오의 유사도 검색


전제 조건:
- 데이터를 의미 있는 벡터 값으로 변환할 수 있어야 합니다.
- 벡터는 Elasticsearch 내부의 NLP 모델을 사용하거나 외부에서 생성할 수 있습니다.
- 문서에 dense_vector 필드 값으로 벡터를 추가합니다.
- 쿼리도 동일한 차원의 벡터로 표현됩니다.


벡터 설계 원칙:
- 유사도 메트릭 기준으로 문서의 벡터가 쿼리 벡터에 가까울수록 더 좋은 매치가 되도록 설계해야 합니다.


필요한 인덱스 권한:
- create_index 또는 manage: dense_vector 필드를 가진 인덱스 생성
- create, index, 또는 write: 생성한 인덱스에 데이터 추가
- read: 인덱스 검색


중요한 포인트:
- kNN 검색은 벡터 공간에서의 유사도를 기반으로 합니다.
- 효과적인 kNN 검색을 위해서는 데이터를 적절한 벡터 표현으로 변환하는 것이 중요합니다.
- 벡터의 차원과 유사도 메트릭은 검색 성능과 정확도에 큰 영향을 미칩니다.

## 벡터 유사도 메트릭(similarity metric)

Dense Vector Field 에서 지정할 수 있는 유사도 검색 파라미터에 대해 알아보자. 

similarity 파라미터:
- kNN 검색에서 사용할 벡터 유사도 메트릭을 지정합니다.
- 옵션이지만, index가 true일 때만 지정할 수 있습니다.
- 기본값은 cosine입니다

l2_norm (L2 거리, 유클리드 거리):
- 벡터 간의 절대적인 거리가 중요할 때
- 데이터 포인트의 실제 위치나 크기가 의미가 있을 때 
- 고차원 데이터에는 유용하지 않음. 
- 이상치 탐지나 클러스터링 작업에서 유용 (데이터 포인트간의 거리 차이로 이상치를 탐색함.)

dot_product: 
- 모든 벡터가 이미 정규화되어 있을 때 (cosine 유사도의 최적화 버전)
- 대규모 데이터셋에서는 계산 효율성이 중요할 수 있으며, 이 경우 dot_product가 유리할 수 있습니다.

dot_product (내적): 사용을 피해야 할 경우:
- 벡터가 정규화되어 있지 않을 때 (길이가 1이 아닌 경우)
- 벡터의 크기 차이가 결과에 영향을 주면 안 될 때
- 음수 값을 포함한 데이터에서 유사도의 해석이 중요할 때
- 데이터의 스케일이 일정하지 않은 경우
- 예: 대규모 텍스트 코퍼스에서의 빠른 유사도 검색, 임베딩 벡터를 사용한 단어 유사성 비교


cosine (코사인 유사도):
- 벡터의 방향이 중요하고 크기는 덜 중요할 때
- 텍스트 문서의 유사성을 비교할 때
- 고차원 데이터에서 작동이 잘 될 때
- 예시: 문서 유사도 검색, 추천 시스템 자연어 처리 작업

cosine (코사인 유사도): 사용을 피해야 할 경우:
- 벡터의 크기(magnitude)가 중요한 의미를 가질 때
- 음수 값을 포함한 데이터의 유사도를 비교할 때 (코사인 유사도는 양수 공간에서만 의미가 있음)
- 벡터의 절대적 위치가 중요한 경우 (예: 지리적 위치 데이터)
- 영벡터(모든 요소가 0인 벡터)를 포함한 데이터셋

max_inner_product:
- 벡터의 방향과 크기가 중요한 의미를 가질 때
- 정규화되지 않은 벡터를 다룰 때
- 양수와 음수 값을 모두 고려해야 할 때 
- 예: 추천 시스템에서 사용자-아이템 상호작용 강도를 고려할 때, 특성의 중요도가 서로 다른 기계학습 모델의 결과를 비교할 때


데이터의 특성: 데이터가 방향만 중요한지, 절대적 크기도 중요한지 고려합니다.
정규화 여부: 데이터가 이미 정규화되어 있다면 dot_product나 cosine이 효율적일 수 있습니다.
계산 효율성: 대규모 데이터셋에서는 계산 효율성이 중요할 수 있으며, 이 경우 dot_product가 유리할 수 있습니다.
벡터 값 범위: 음수 지원하는지 

dot_product vs cosine 
- 둘 다 벡터의 방향이 더 중요
- cosine 은 -1 ~ 1 까지 지원, dot_product 는 양수만 지원. 그래서 cosine 이 더 넓은 유사도 범위를 할 수 있음. 
- dot_product 는 cosine 보다 계산이 간단해서 대규모 데이터에서 더 빠름. 
- dot_product 는 정규화 벡터에서만 동작. cosine 은 알아서 정규화해줌. 


max_inner_product vs l2_norm 
- max_inner_product 는 벡터의 방향과 크기도 고려하는 반면에, l2_norm 은 순수 거리만을 측정한다. 




## kNN Methods 

Elasticsearch의 두 가지 kNN 검색 방법: 
- Approximate kNN: 
  - 낮은 지연 시간을 제공하지만, 인덱싱이 느리고 완벽한 정확도를 보장하지 않습니다.

- brute-force kNN:  
  - 정확한 결과를 보장하지만, 대규모 데이터셋에서는 확장성이 떨어집니다.

사용 시 고려사항:
- 데이터셋의 크기
- 요구되는 정확도 수준
- 허용 가능한 검색 지연 시간
- 인덱싱 속도의 중요성

브루트 포스 kNN의 활용:
- 데이터를 작은 부분집합으로 필터링할 수 있는 경우, 이 방법으로도 좋은 검색 성능을 얻을 수 있습니다.





## Approximate kNN

리소스 요구사항: 
- 근사 kNN 검색은 특별한 리소스 요구사항이 있습니다.
- 모든 벡터 데이터가 노드의 페이지 캐시에 맞아야 효율적입니다.
- 구성 및 크기 조정에 대한 튜닝 가이드를 참조하는 것이 중요합니다.

kNN 쿼리 실행: 
- 'knn' 옵션을 사용하여 검색을 실행합니다.
- 검색 파라미터로는 field, query_vector, k, num_candidates 등이 있습니다.
- 문서의 _score는 쿼리 벡터와 문서 벡터 간의 유사도에 의해 결정됩니다.

```json
POST image-index/_search
{
  "knn": {
    "field": "image-vector",
    "query_vector": [-5, 9, -12],
    "k": 10,
    "num_candidates": 100
  },
  "fields": [ "title", "file-type" ]
}

``` 

매핑 요구사항:
- dense_vector 필드를 명시적으로 매핑해야 합니다.
- similarity 값을 지정해야 합니다 (기본값은 cosine).
- 예시에서는 'l2_norm' 유사도 메트릭을 사용합니다.

```json
PUT image-index
{
  "mappings": {
    "properties": {
      "image-vector": {
        "type": "dense_vector",
        "dims": 3,
        "similarity": "l2_norm"
      },
      "title-vector": {
        "type": "dense_vector",
        "dims": 5,
        "similarity": "l2_norm"
      },
      "title": {
        "type": "text"
      },
      "file-type": {
        "type": "keyword"
      }
    }
  }
}
```

버전 호환성:
- 근사 kNN 검색 지원은 Elasticsearch 8.0 버전에서 추가되었습니다.
- 8.0 이전 버전에서 생성된 인덱스의 경우, 근사 kNN 검색을 지원하려면 index: true 옵션으로 데이터를 재인덱싱해야 합니다.









## Tune approximate kNN for speed or accuracy

kNN 검색 과정:
- 각 샤드에서 'num_candidates' 수만큼의 근사 최근접 이웃 후보를 찾습니다.
- 이 후보 벡터들과 쿼리 벡터 간의 유사도를 계산합니다.
- 각 샤드에서 가장 유사한 k개의 결과를 선택합니다.
- 모든 샤드의 결과를 병합하여 전체 상위 k개의 최근접 이웃을 반환합니다.

num_candidates 파라미터의 역할:
- 이 파라미터는 정확도와 검색 속도 사이의 균형을 조절합니다.


num_candidates 값 증가:
- 장점: 더 정확한 결과를 얻을 수 있습니다.
- 단점: 검색 속도가 느려집니다.
- 이유: 각 샤드에서 더 많은 후보를 고려하므로, 진정한 상위 k개 최근접 이웃을 찾을 확률이 높아집니다.


num_candidates 값 감소:
- 장점: 검색 속도가 빨라집니다.
- 단점: 결과의 정확도가 떨어질 수 있습니다.
- 이유: 각 샤드에서 고려하는 후보 수가 줄어들어 처리 시간이 단축됩니다.




## Approximate kNN using byte vectors

바이트 벡터 지원:
- 근사 kNN 검색 API는 float 값 벡터 외에도 바이트 값 벡터를 지원합니다.
- dense_vector 필드에 element_type을 'byte'로 설정하고 인덱싱을 활성화해야 합니다.

```json 
PUT byte-image-index
{
  "mappings": {
    "properties": {
      "byte-image-vector": {
        "type": "dense_vector",
        "element_type": "byte",
        "dims": 2
      },
      "title": {
        "type": "text"
      }
    }
  }
}
```

데이터 인덱싱:
- 모든 벡터 값은 -128에서 127 사이의 정수여야 합니다.

주요 포인트:
- 바이트 벡터는 float 벡터에 비해 메모리 사용량을 줄일 수 있습니다.
- 값의 범위가 제한되어 있으므로(-128에서 127), 이 범위 내에서 데이터를 적절히 정규화해야 합니다.


## Byte quantized kNN search 

바이트 양자화의 목적:
- float 벡터를 제공하면서도 바이트 벡터의 메모리 절약 효과를 얻을 수 있습니다.
- float 벡터를 내부적으로 바이트 벡터로 인덱싱하지만, 원본 float 벡터도 인덱스에 유지합니다.

기본 인덱스 타입:
- dense_vector의 기본 인덱스 타입은 'int8_hnsw'입니다.

인덱스 매핑:
- dense_vector 필드에 'int8_hnsw' 인덱스 타입을 지정합니다.
- element_type은 'float'으로 설정합니다.

```json 
PUT quantized-image-index
{
  "mappings": {
    "properties": {
      "image-vector": {
        "type": "dense_vector",
        "element_type": "float",
        "dims": 2,
        "index": true,
        "index_options": {
          "type": "int8_hnsw"
        }
      },
      "title": {
        "type": "text"
      }
    }
  }
}
```

'knn' 옵션을 사용하여 검색을 실행합니다.
- 검색 시 float 벡터가 자동으로 바이트 벡터로 양자화됩니다.


```json
POST quantized-image-index/_search
{
  "knn": {
    "field": "image-vector",
    "query_vector": [0.1, -2],
    "k": 10,
    "num_candidates": 100
  },
  "fields": [ "title" ]
}
```

재점수화(Rescoring):
- 원본 float 벡터를 사용하여 상위 결과의 점수를 재계산할 수 있습니다.
- 상위 k개 결과에 대해서만 원본 float 벡터를 사용하여 재점수화합니다.
- 이를 통해 빠른 검색과 정확한 점수 계산의 장점을 모두 얻을 수 있습니다.


```json
POST quantized-image-index/_search
{
  "knn": {
    "field": "image-vector",
    "query_vector": [0.1, -2],
    "k": 15,
    "num_candidates": 100
  },
  "fields": [ "title" ],
  "rescore": {
    "window_size": 10,
    "query": {
      "rescore_query": {
        "script_score": {
          "query": {
            "match_all": {}
          },
          "script": {
            "source": "cosineSimilarity(params.query_vector, 'image-vector') + 1.0",
            "params": {
              "query_vector": [0.1, -2]
            }
          }
        }
      }
    }
  }
}
```




## Filtered kNN search

필터링된 kNN 검색:
- kNN 검색 API는 필터를 사용하여 검색 범위를 제한할 수 있습니다.
- 검색은 필터 쿼리와 일치하는 문서 중에서 상위 k개의 문서를 반환합니다.

검색 요청 예시:
- 'image-vector' 필드에 대해 근사 kNN 검색을 수행합니다.
- 'file-type' 필드를 기준으로 필터링합니다 (이 예에서는 'png' 파일만 검색).

```json
POST image-index/_search
{
  "knn": {
    "field": "image-vector",
    "query_vector": [54, 10, -2],
    "k": 5,
    "num_candidates": 50,
    "filter": {
      "term": {
        "file-type": "png"
      }
    }
  },
  "fields": ["title"],
  "_source": false
}
```

필터 적용 방식:
- 필터는 근사 kNN 검색 중에 적용됩니다.
- 이는 k개의 일치하는 문서를 확실히 반환하기 위함입니다.

후처리 필터링과의 차이:
- 후처리 필터링은 kNN 검색 완료 후 필터를 적용합니다.
- 후처리 필터링의 단점: 충분한 일치 문서가 있어도 k개 미만의 결과를 반환할 수 있습니다.

주의사항:
- 필터가 너무 제한적이면 k개의 결과를 찾지 못할 수 있습니다.
- num_candidates 값을 적절히 설정하여 검색의 정확성과 속도를 조절해야 합니다.







## Approximate kNN search and filtering

일반적인 쿼리와 필터링 쿼리와의 차이:
- ES 에서는 일반적인 쿼리에서는 더 제한적인 필터가 보통 더 빠른 쿼리 실행을 의미합니다.
- 그러나 HNSW 인덱스를 사용한 근사 kNN 검색에서는 필터 적용이 오히려 성능을 저하시킬 수 있습니다.


성능 저하의 이유:
- HNSW 그래프 검색 시 필터 기준을 만족하는 num_candidates를 얻기 위해 추가적인 탐색이 필요합니다.
- 추가적인 오버헤드가 있는거지. 일반적인 필터는 탐색 범위를 줄이고 가는 반면에 여기서는 HNSW 탐색 하면서 필터를 적용하다보니까. 



Lucene의 성능 최적화 전략:
- Lucene은 세그먼트 별로 다음 전략을 구현하여 성능 저하를 방지합니다:
  - 필터링된 문서 수가 num_candidates 이하인 경우:
    - HNSW 그래프 검색을 우회해서, 필터링된 문서에 대해 브루트 포스 검색을 수행합니다.
  - HNSW 그래프 탐색 중 특정 조건 만족 시: 
    - 탐색된 노드 수가 필터를 만족하는 문서 수를 초과하면, 그래프 탐색을 중단하고 필터링된 문서에 대해 브루트 포스 검색으로 전환합니다.

필터링 + HNSW 그래프 검색 매커니즘: 
- 필터링으로 통과한 문서의 집합을 구함. 
- num_candidated 수만큼을 구하기 위해서 HNSW 인덱스를 타서 검색을 함. 여기서 마지막 노드가 필터링을 통과하지 못한다면 추가 탐색이 계속적으로 발생할 수 있음. 그래서 필터링이 성능 저하를 일으킴. 


Lucene 성능 최적화 해석: 
- num_candidated 만큼 최소 HNSW 탐색이 이뤄지기 때문에, 필터링 된 문서가 num_candidated 보다 이하라면 브루트 포스 접근을 하는것. 
- 필터링된 문서의 수만큼 브루트 포스를 하게 되면 최대 시간 복잡도는 계산할 수 있음. 근데 num_candidate 가 필터링 문서보다 작다면 HNSW 탐색으로 더 빠르게 찾아낼 수도 있는 가능성이 있음. 근데 이게 필터링된 문서 수보다 많이 탐색을 해야한다면 그냥 브루트 포스 접근이 나은 것이었으니 이 방식으로 하는 것. 

사용 시 고려사항:
- num_candidates 값을 적절히 설정하는 것이 중요합니다.
- 필터의 선택성을 고려하여 쿼리를 설계해야 합니다.
- 대규모 데이터셋에서는 필터링과 kNN 검색의 균형을 잘 맞추어야 합니다.











## Combine approximate kNN with other features

하이브리드 검색:
- kNN 옵션과 일반 쿼리를 함께 사용하여 하이브리드 검색을 수행할 수 있습니다.

예시 쿼리 
- 텍스트 매치 쿼리("mountain lake")와 벡터 kNN 검색을 결합합니다.
- 각 부분에 boost 값을 적용하여 가중치를 조절합니다.

```json 
POST image-index/_search
{
  "query": {
    "match": {
      "title": {
        "query": "mountain lake",
        "boost": 0.9
      }
    }
  },
  "knn": {
    "field": "image-vector",
    "query_vector": [54, 10, -2],
    "k": 5,
    "num_candidates": 50,
    "boost": 0.1
  },
  "size": 10
}
```

결과 결합 방식:
- kNN 결과와 쿼리 결과는 disjunction(OR) 방식으로 결합됩니다.

점수 계산:
- 각 히트의 점수는 kNN 점수와 쿼리 점수의 가중 합으로 계산됩니다.
- 예: score = 0.9 * match_score + 0.1 * knn_score






## Perform semantic search

의미론적 검색의 개념:
- 검색어의 문자 그대로의 일치가 아닌, 검색 쿼리의 의도와 문맥적 의미에 기반하여 결과를 검색합니다.


작동 원리:
- 사전에 배포된 텍스트 임베딩 모델을 사용합니다.
- 입력 쿼리 문자열을 밀집 벡터(dense vector)로 변환합니다.
- 이 벡터를 동일한 모델로 생성된 밀집 벡터가 저장된 인덱스에 대해 검색합니다.


의미론적 검색 수행을 위한 요구사항:
- 검색 대상 데이터의 밀집 벡터 표현이 포함된 인덱스가 필요합니다.
- 검색에 사용하는 텍스트 임베딩 모델은 입력 데이터의 벡터 생성에 사용한 모델과 동일해야 합니다.
- 텍스트 임베딩 NLP 모델 배포가 시작되어 있어야 합니다.

쿼리 구조:
- query_vector_builder 객체를 사용하여 배포된 텍스트 임베딩 모델을 참조합니다.
- model_text 파라미터에 검색 쿼리를 제공합니다.

```json
{
  "knn": {
    "field": "dense-vector-field",
    "k": 10,
    "num_candidates": 100,
    "query_vector_builder": {
      "text_embedding": { 
        "model_id": "my-text-embedding-model", 
        "model_text": "The opposite of blue" 
      }
    }
  }
}
```

추가 정보:
- 훈련된 모델을 배포하고 텍스트 임베딩을 생성하는 방법에 대한 자세한 정보는 별도의 예제를 참조
- https://www.elastic.co/guide/en/machine-learning/8.14/ml-nlp-text-emb-vector-search-example.html







## Search multiple kNN fields

여러 kNN(k-Nearest Neighbors) 벡터 필드를 동시에 검색하는 방법 

다중 kNN 필드 검색:
- 하나 이상의 kNN 벡터 필드를 동시에 검색할 수 있습니다.
- 이는 하이브리드 검색의 확장된 형태입니다.

예시 쿼리 구조:
- 텍스트 매치 쿼리("mountain lake")
- 두 개의 kNN 검색 (image-vector와 title-vector)
- 각 부분에 대한 boost 값 지정


결과 결합 방식:
- 여러 kNN 엔트리와 쿼리 매치는 disjunction(OR) 방식으로 결합됩니다.
- 각 벡터 필드의 상위 k 결과는 모든 인덱스 샤드에 걸친 전역 최근접 이웃을 나타냅니다.


점수 계산:
- 각 문서의 점수는 텍스트 매치 점수와 각 kNN 검색의 점수를 가중 합산하여 계산됩니다.
- 예시: score = 0.9 * match_score + 0.1 * knn_score_image-vector + 0.5 * knn_score_title-vector


```json 
POST image-index/_search
{
  "query": {
    "match": {
      "title": {
        "query": "mountain lake",
        "boost": 0.9
      }
    }
  },
  "knn": [ {
    "field": "image-vector",
    "query_vector": [54, 10, -2],
    "k": 5,
    "num_candidates": 50,
    "boost": 0.1
  },
  {
    "field": "title-vector",
    "query_vector": [1, 20, -52, 23, 10],
    "k": 10,
    "num_candidates": 10,
    "boost": 0.5
  }],
  "size": 10
}
```

주요 이점:
- 다양한 유형의 벡터 데이터를 동시에 고려할 수 있습니다.
- 텍스트 기반 검색과 여러 벡터 기반 검색을 결합하여 더 풍부한 검색 결과를 제공합니다.
- 각 검색 구성요소에 대한 가중치(boost)를 조정하여 검색 결과의 우선순위를 세밀하게 제어할 수 있습니다.




## Search kNN with expected similarity

Elasticsearch의 kNN(k-Nearest Neighbors) 검색에서 기대 유사도(expected similarity)를 사용하는 방법에 대해 설명

kNN의 한계점:
- kNN은 항상 k개의 최근접 이웃을 반환하려고 합니다.
- 필터와 함께 사용할 때, 관련 없는 문서만 남을 수 있습니다.


similarity 파라미터:
- kNN 절에서 사용 가능한 새로운 파라미터입니다.
- 벡터가 매치로 간주되기 위한 최소 유사도를 지정합니다.

similarity를 사용한 kNN 검색 흐름:
- 사용자가 제공한 필터 쿼리 적용
- 벡터 공간에서 k개의 벡터 탐색
- 구성된 similarity보다 멀리 있는 벡터는 반환하지 않음

similarity와 _score의 관계:
- similarity는 _score로 변환되기 전의 실제 유사도입니다.
- 각 유사도 메트릭에 대한 _score 변환 공식이 제공됩니다.
  - l2_norm: sqrt((1 / _score) - 1)
  - cosine: (2 * _score) - 1
  - dot_product: (2 * _score) - 1
  - max_inner_product:
    - _score < 1: 1 - (1 / _score)
    - _score >= 1: _score - 1

```json

POST image-index/_search
{
  "knn": {
    "field": "image-vector",
    "query_vector": [1, 5, -20],
    "k": 5,
    "num_candidates": 50,
    "similarity": 36,
    "filter": {
      "term": {
        "file-type": "png"
      }
    }
  },
  "fields": ["title"],
  "_source": false
}
```

주요 이점:
- 관련성 없는 결과를 효과적으로 필터링할 수 있습니다.
- 유사도에 기반한 더 정확한 검색 결과를 얻을 수 있습니다.


사용 시 고려사항:
- similarity 값을 적절히 설정하는 것이 중요합니다. (유사도 메트릭도 추가로 고려해야함.)
- 데이터의 특성과 벡터 공간의 분포를 이해해야 합니다.
- 필터와 similarity를 함께 사용할 때 검색 결과가 없을 수 있음을 인지해야 합니다.









## Nested kNN Search

텍스트 청킹(Chunking)의 필요성:
- 텍스트가 특정 모델의 토큰 제한을 초과할 때 청킹이 필요합니다.
- 개별 청크에 대한 임베딩을 생성해야 합니다.

기존에 텍스트 청킹 방법: 
- 청킹 텍스트에다가 메타 데이터로 상위 문서를 포함시켰음. 그래서 저장 비용이 두 배로 든다. 

ES 에서는 Nested 와 Dense Vector 의 결합으로 데이터를 저장할 수 있음. 
- 문서 내용 전체를 인덱싱하고, 청킹 부분 (Nested, paragraph) 부분만 각각 저장하면 됨. 

다음과 같이 매핑을 만들어야한다. 
- 'passage_vectors' 인덱스를 생성합니다.
- 'full_text', 'creation_time'과 같은 상위 레벨 필드를 포함합니다.
- 'paragraph'라는 nested 타입 필드를 정의합니다.
- 'vector': dense_vector 타입, HNSW 인덱스 옵션 사용
- 'text': 텍스트 필드, 인덱싱되지 않음


```json
PUT passage_vectors
{
    "mappings": {
        "properties": {
            "full_text": {
                "type": "text"
            },
            "creation_time": {
                "type": "date"
            },
            "paragraph": {
                "type": "nested",
                "properties": {
                    "vector": {
                        "type": "dense_vector",
                        "dims": 2,
                        "index_options": {
                            "type": "hnsw"
                        }
                    },
                    "text": {
                        "type": "text",
                        "index": false
                    }
                }
            }
        }
    }
}
``` 

색인도 다음 방법처럼 벌크로 하면 됨. 

```json
POST passage_vectors/_bulk?refresh=true
{ "index": { "_id": "1" } }
{ "full_text": "first paragraph another paragraph", "creation_time": "2019-05-04", "paragraph": [ { "vector": [ 0.45, 45 ], "text": "first paragraph", "paragraph_id": "1" }, { "vector": [ 0.8, 0.6 ], "text": "another paragraph", "paragraph_id": "2" } ] }
{ "index": { "_id": "2" } }
{ "full_text": "number one paragraph number two paragraph", "creation_time": "2020-05-04", "paragraph": [ { "vector": [ 1.2, 4.5 ], "text": "number one paragraph", "paragraph_id": "1" }, { "vector": [ -1, 42 ], "text": "number two paragraph", "paragraph_id": "2" } ] }
```


검색 방법: 

```json
POST passage_vectors/_search
{
    "fields": ["full_text", "creation_time"],
    "_source": false,
    "knn": {
        "query_vector": [
            0.45,
            45
        ],
        "field": "paragraph.vector",
        "k": 2,
        "num_candidates": 2
    }
}
```

검색 결과의 특징:
- 총 4개의 벡터가 있지만, 2개의 문서만 반환됩니다.
- Nested dense_vectors에 대한 kNN 검색은 항상 상위 레벨 문서에 대해 결과를 다양화합니다.
- 'k'개의 상위 레벨 문서가 반환되며, 각 문서는 가장 가까운 패시지 벡터에 의해 점수가 매겨집니다.


```json
{
    "took": 4,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2,
            "relation": "eq"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "passage_vectors",
                "_id": "1",
                "_score": 1.0,
                "fields": {
                    "creation_time": [
                        "2019-05-04T00:00:00.000Z"
                    ],
                    "full_text": [
                        "first paragraph another paragraph"
                    ]
                }
            },
            {
                "_index": "passage_vectors",
                "_id": "2",
                "_score": 0.9997144,
                "fields": {
                    "creation_time": [
                        "2020-05-04T00:00:00.000Z"
                    ],
                    "full_text": [
                        "number one paragraph number two paragraph"
                    ]
                }
            }
        ]
    }
}
```

## Nested kNN Search with Inner hits

Nested Search 에서 Inner Hits 를 이용하는 법을 다룸. 

Inner Hits: 
- 매치된 문서에서 가장 가까운 패시지(passage)를 추출하기 위해 사용됩니다.

Inner Hits 사용 방법:
- kNN 절에 inner_hits를 추가하여 사용합니다.

장점:
- 문서 전체가 아닌 가장 관련 있는 패시지를 정확히 식별할 수 있습니다.
- 긴 문서에서 특정 부분을 효과적으로 검색하고 추출할 수 있습니다.
- 상위 레벨 문서 컨텍스트와 함께 세부적인 매치 정보를 얻을 수 있습니다.


검색 예시 inner_hits:
- _source: false로 설정하여 nested 객체의 원본을 반환하지 않습니다.
- fields: 반환할 nested 필드를 지정합니다 ("paragraph.text").
- size: 각 문서당 반환할 inner hit의 수를 지정합니다 (여기서는 1).


```json
POST passage_vectors/_search
{
    "fields": [
        "creation_time",
        "full_text"
    ],
    "_source": false,
    "knn": {
        "query_vector": [
            0.45,
            45
        ],
        "field": "paragraph.vector",
        "k": 2,
        "num_candidates": 2,
        "inner_hits": {
            "_source": false,
            "fields": [
                "paragraph.text"
            ],
            "size": 1
        }
    }
}
```

Inner hits 검색 결과: 
```json 
{
    "took": 4,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2,
            "relation": "eq"
        },
        "max_score": 1.0,
        "hits": [
            {
                "_index": "passage_vectors",
                "_id": "1",
                "_score": 1.0,
                "fields": {
                    "creation_time": [
                        "2019-05-04T00:00:00.000Z"
                    ],
                    "full_text": [
                        "first paragraph another paragraph"
                    ]
                },
                "inner_hits": {
                    "paragraph": {
                        "hits": {
                            "total": {
                                "value": 2,
                                "relation": "eq"
                            },
                            "max_score": 1.0,
                            "hits": [
                                {
                                    "_index": "passage_vectors",
                                    "_id": "1",
                                    "_nested": {
                                        "field": "paragraph",
                                        "offset": 0
                                    },
                                    "_score": 1.0,
                                    "fields": {
                                        "paragraph": [
                                            {
                                                "text": [
                                                    "first paragraph"
                                                ]
                                            }
                                        ]
                                    }
                                }
                            ]
                        }
                    }
                }
            },
            {
                "_index": "passage_vectors",
                "_id": "2",
                "_score": 0.9997144,
                "fields": {
                    "creation_time": [
                        "2020-05-04T00:00:00.000Z"
                    ],
                    "full_text": [
                        "number one paragraph number two paragraph"
                    ]
                },
                "inner_hits": {
                    "paragraph": {
                        "hits": {
                            "total": {
                                "value": 2,
                                "relation": "eq"
                            },
                            "max_score": 0.9997144,
                            "hits": [
                                {
                                    "_index": "passage_vectors",
                                    "_id": "2",
                                    "_nested": {
                                        "field": "paragraph",
                                        "offset": 1
                                    },
                                    "_score": 0.9997144,
                                    "fields": {
                                        "paragraph": [
                                            {
                                                "text": [
                                                    "number two paragraph"
                                                ]
                                            }
                                        ]
                                    }
                                }
                            ]
                        }
                    }
                }
            }
        ]
    }
}
```




## Indexing considerations

근사 kNN(k-Nearest Neighbors) 검색을 위한 인덱싱 고려사항에 대해 다룸. 

HNSW 그래프 저장:
- Elasticsearch는 각 세그먼트의 dense vector 값을 HNSW(Hierarchical Navigable Small World) 그래프로 저장합니다.

인덱싱 시간:
- 근사 kNN 검색을 위한 벡터 인덱싱은 상당한 시간이 소요될 수 있습니다.
- HNSW 그래프 구축이 계산 비용이 높기 때문입니다.


클라이언트 요청 타임아웃:
- 인덱스 및 벌크 요청에 대한 클라이언트 요청 타임아웃을 증가시켜야 할 수 있습니다.

성능 튜닝 가이드:
- 인덱싱 성능과 검색 성능에 영향을 미치는 인덱스 구성에 대한 중요한 지침이 있습니다.
- 참고: https://www.elastic.co/guide/en/elasticsearch/reference/current/tune-knn-search.html


HNSW 알고리즘 파라미터:
- 검색 시간 튜닝 파라미터 외에도 인덱스 시간 파라미터가 있습니다.
- 이 파라미터들은 그래프 구축 비용, 검색 속도, 정확도 사이의 균형을 조절합니다.


index_options 설정:
- dense_vector 매핑 설정 시 index_options 인자를 사용하여 이러한 파라미터를 조정할 수 있습니다.
- type: HNSW 알고리즘 사용을 지정
- m: 그래프의 각 노드가 가질 수 있는 최대 연결 수 (높을수록 더 정확하지만 인덱싱이 느려짐) (기본값은 16)
- ef_construction: 그래프에 새로운 노드를 추가할 때 고려해야하는 후보 노드의 수임. (높을수록 더 정확하지만 인덱싱이 느려짐) (기본값은 100)


```json 
PUT image-index
{
  "mappings": {
    "properties": {
      "image-vector": {
        "type": "dense_vector",
        "dims": 3,
        "similarity": "l2_norm",
        "index_options": {
          "type": "hnsw",
          "m": 32,
          "ef_construction": 100
        }
      }
    }
  }
}
```






## Limitations for approximate kNN search

근사 kNN(k-Nearest Neighbors) 검색의 제한사항에 대해 다룸. 

크로스 클러스터 검색 제한:
- kNN 검색을 크로스 클러스터 검색에서 사용할 때, 'ccs_minimize_roundtrips' 옵션은 지원되지 않습니다.
- 이는 크로스 클러스터 검색 최적화의 일부 기능이 kNN 검색과 호환되지 않음을 의미합니다.

HNSW 알고리즘의 특성:
- Elasticsearch는 효율적인 kNN 검색을 위해 HNSW(Hierarchical Navigable Small World) 알고리즘을 사용합니다.
- HNSW는 근사 방법으로, 대부분의 kNN 알고리즘과 마찬가지로 결과의 정확성을 일부 희생하여 검색 속도를 향상시킵니다.
- 이는 반환된 결과가 항상 진정한 k개의 가장 가까운 이웃이 아닐 수 있음을 의미합니다.



## Exact kNN 

여기서는 Elasticsearch에서 정확한 kNN(k-Nearest Neighbors) 검색을 수행하는 방법을 다룸. 이전까지는 근사 kNN 에 대해 다루었다면. 

정확한 kNN 검색 방법:
- script_score 쿼리와 벡터 함수를 사용합니다.

인덱스 매핑:
- dense_vector 필드를 명시적으로 매핑합니다.
- 근사 kNN을 사용하지 않을 경우, index 옵션을 false로 설정하여 인덱싱 속도를 향상시킵니다.

```json
PUT product-index
{
  "mappings": {
    "properties": {
      "product-vector": {
        "type": "dense_vector",
        "dims": 5,
        "index": false
      },
      "price": {
        "type": "long"
      }
    }
  }
}
```


검색 쿼리 구조:
- script_score 쿼리를 사용합니다.
- 벡터 함수(여기서는 cosineSimilarity)를 스크립트에 포함시킵니다.


성능 최적화:
- filter 쿼리를 사용하여 벡터 함수에 전달되는 문서 수를 제한합니다.
- 모든 문서를 매치하는 것은 검색 지연 시간을 크게 증가시킬 수 있습니다.


```json
POST product-index/_search
{
  "query": {
    "script_score": {
      "query" : {
        "bool" : {
          "filter" : {
            "range" : {
              "price" : {
                "gte": 1000
              }
            }
          }
        }
      },
      "script": {
        "source": "cosineSimilarity(params.queryVector, 'product-vector') + 1.0",
        "params": {
          "queryVector": [-0.5, 90.0, -10, 14.8, -156.0]
        }
      }
    }
  }
}
```

주요 특징:
- 정확성: 근사 방법과 달리 정확한 kNN 결과를 제공합니다.
- 유연성: 다양한 벡터 함수와 필터링 조건을 조합할 수 있습니다.
- 성능 고려: 대규모 데이터셋에서는 느릴 수 있으므로 적절한 필터링이 중요합니다.





## Hybrid Search: Combined Full-Text and kNN Results
You have now seen two different approaches to search a collection of documents, each with its own particular benefits. If one of these methods matches your needs then you don't need anything else, but in many cases each method of searching returns valuable results that the other method would miss, so the best option is to offer a combined result set.

For these cases, Elasticsearch offers Reciprocal Rank Fusion, an algorithm that combines results from two or more lists into a single list.



## How RRF Works
Elasticsearch integrates the RRF algorithm into the search query. Consider the following example, which has query and knn sections to request full-text and vector searches respectively, and a rrf section that combines them into a single result list.


```python
self.es.search(
    query={
        # full-text search query here
    },
    knn={
        # vector search query here
    },
    rank={
        "rrf": {}
    }
)

```

While RRF works fairly well for short lists of results without any configuration, there are some parameters that can be tuned to provide the best results. Consult the documentation to learn about these in detail.



## Reciprocal rank fusion

참고: https://youngerjesus.tistory.com/148


In [31]:
import pprint
import json

class Search:
    def __init__(self):
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.es = Elasticsearch('http://localhost:9200')
        client_info = self.es.info()
        print('Connected to Elasticsearch!')
        # pprint(client_info.body)

    def create_index(self):
        self.es.indices.delete(index='my_documents', ignore_unavailable=True)
        self.es.indices.create(index='my_documents', mappings={
            'properties': {
                'embedding': {
                    'type': 'dense_vector',
                }
            }
        })

    def insert_document(self, document):
        return self.es.index(index='my_documents', document={
            **document,
            'embedding': self.get_embedding(document['summary']),
        })
    
    def insert_documents(self, documents):
        operations = []
        for document in documents:
            operations.append({'index': {'_index': 'my_documents'}})
            operations.append({
                **document,
                'embedding': self.get_embedding(document['summary']),
            })
        return self.es.bulk(operations=operations)


    def reindex(self):
        self.create_index()
        with open('data.json', 'rt') as f:
            documents = json.loads(f.read())
        return self.insert_documents(documents)
    
    
    def get_embedding(self, text):
        return self.model.encode(text)
    
    def extract_filters(self, query):
        # This is a placeholder. Implement your filter extraction logic here.
        return {}, query

    def hybrid_search(self, query, from_=0, size=5):
        filters, parsed_query = self.extract_filters(query)

        if parsed_query:
            search_query = {
                'must': {
                    'multi_match': {
                        'query': parsed_query,
                        'fields': ['name', 'summary', 'content'],
                    }
                }
            }
        else:
            search_query = {
                'must': {
                    'match_all': {}
                }
            }

        results = self.es.search(
            index='my_documents',
            query={
                'bool': {
                    **search_query,
                }
            },
            knn={
                'field': 'embedding',
                'query_vector': self.get_embedding(parsed_query),
                'k': 10,
                'num_candidates': 50,
            },
            rank={
                'rrf': {}
            },
            aggs={
                'year-agg': {
                    'date_histogram': {
                        'field': 'updated_at',
                        'calendar_interval': 'year',
                        'format': 'yyyy',
                    },
                },
            },
            size=size,
            from_=from_,
        )

        aggs = {
            'Year': {
                bucket['key_as_string']: bucket['doc_count']
                for bucket in results['aggregations']['year-agg']['buckets']
                if bucket['doc_count'] > 0
            },
        }

        return {
            'hits': results['hits']['hits'],
            'total': results['hits']['total']['value'],
            'aggs': aggs
        }
    

In [32]:
search = Search()

/Users/jeongmin/PycharmProjects/python-skills-bootcamp/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Connected to Elasticsearch!


In [ ]:
search.reindex()

In [33]:
results = search.hybrid_search("tesla")

print(results)

{'hits': [{'_index': 'my_documents', '_id': '_hPcpZABMnum9rmLig26', '_score': None, '_rank': 1, '_ignored': ['content.keyword'], '_source': {'name': 'Huge process.', 'url': 'http://www.fitzpatrick.net/', 'summary': 'Lawyer pay Congress indeed owner. Easy face boy feel performance history car.', 'content': 'Whose front on well gun happy. Blood other hear need write turn prepare huge.\nAs positive own production. Heart too government woman draw how particular. Team us act seek.\nWord wind what management. Live officer over radio. Traditional player subject religious turn.\nOut bank car shoulder will available. Upon mention report major. West toward oil shake help operation.\nNor reveal wide and never focus share.\nVisit street ask gas show single pay matter.\nLeg strong name chance travel skill. Game president type would speech road north moment. Cut beyond early say.\nBehind together series suggest state campaign. Able two begin rest instead teacher.\nToo phone save yeah threat read sou